In [1]:
import numpy as np
import pandas as pd 
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.table import Table
import glob

%matplotlib inline

In [2]:
typeinfo = {
    "Ia":[1,1],
    "Ia-norm":[1,2],
    "Ia-91T":[1,3],
    "Ia-91bg":[1,4],
    "Ia-csm":[1,5],
    "Ia-02cx":[1,6],
    "Ia-03fg":[1,7],
    "Ib":[2,1],
    "Ib-norm":[2,2],
    "IIb":[2,3],
    "Ibn":[2,4],
    "Ic":[3,1],
    "Ic-norm":[3,2],
    "Ic-BL":[3,3],
    "Ic-SLSN":[3,4],
    "II":[4,1],
    "II-norm":[4,2],
    "IIn":[4,3],
    "NotSN":[5,1],
    "AGN":[5,2],
    "Gal":[5,3],
    "LBV":[5,4],
    "M-star":[5,5],
    "QSO":[5,6],
    "C-star":[5,7],
}

In [3]:
Changes = Table.from_pandas(pd.read_csv("/home/xhall/Documents/SNID/changes.csv"))

In [4]:
Changes

snname,old_type,new_type
str12,str6,str7
sn2005ek,Ic-pec,Ic-norm
sn2005la,Ib-pec,Ibn
sn1987A,II-pec,II-norm
sn2006gz,Ia-pec,Ia-03fg
sn2005hk,Ia-pec,Ia-02cx
snls03D3bb,Ia-pec,Ia-03fg
sn2008ae,Ia-pec,Ia-02cx
sn2008D,Ib-pec,Ib-norm
sn2002cx,Ia-pec,Ia-02cx


In [15]:
counter = 0
for i in Changes:
    f = open("/home/xhall/Documents/SNID/templates-2.0/" + i["snname"] + ".lnw")
    lines = f.readlines()
    string = lines[0]
    start = string.find(i["old_type"])
    if(start != -1):
        replace = i["new_type"]
        back_string = string[start::]
        back_string_len = len(back_string)
        numlist = typeinfo[replace]
        numstring = str(numlist[0]) + "  " + str(numlist[1])
        back_string_new = str(replace) + " " * (back_string_len - len(numstring) - len(replace) - 1) + numstring + "\n"
        forward_string = string[0:start]
        document = forward_string + back_string_new
        for j in lines[1::]:
            document += j

        newfile = open("/home/xhall/Documents/SNID/templates-2.0/" + i["snname"] + ".lnw", "w+")
        newfile.write(document)
        counter += 1

In [13]:
document

'    6 1024   2500.00  10000.00      8     sn2005ek      -9.99  Ic-norm     3  2\n'

In [3]:
templates = glob.glob("/home/xhall/Documents/SNID/templates-2.0/*.lnw")

In [4]:
TemplatesClass = Table(
                    names=("Template", "Class"
                    ),
                    meta={"name": "Template Types"},
                    dtype=("U64", "U64"
                          )
                    )

for i in templates:
    f = open(i)
    lines = f.readlines()
    row = [i.split("/")[-1].split(".")[0],lines[0].split()[-3]]
    TemplatesClass.add_row(row)

In [5]:
TemplatesClass.to_pandas().to_csv("/home/xhall/Documents/NewZTF/Template_Class.csv", index = False)

In [14]:
TemplatesClass[TemplatesClass["Class"]=="II"]["Template"]

sn07it
sn09au
sn06Y
sn09bz
sn09ao
sn06iw
sn04er
sn06bc
sn06ee
sn99br
sn04fc


In [11]:
for i in np.unique(TemplatesClass["Class"]): print(i)

AGN
C-star
Gal
II
II-norm
IIb
IIn
Ia-02cx
Ia-03fg
Ia-91T
Ia-91bg
Ia-csm
Ia-norm
Ib-norm
Ibn
Ic-BL
Ic-SLSN
Ic-norm
LBV
M-star
QSO


In [7]:
PreferredTemplatesClass = Table(
                    names=("Template", "old_type", "new_type"
                    ),
                    meta={"name": "Template Types"},
                    dtype=("U64", "U64", "U64"
                          )
                    )
for i in templates:
    f = open(i)
    lines = f.readlines()
    row = [i.split("/")[-1].split(".")[0]]
    sntype = lines[0].split()[-3]
    row.append(sntype)
    if(sntype == "Ic-broad"):
        sntype = "Ic-BL"
    elif(sntype == "II"):
        sntype = "II-norm"
    elif(sntype == "IIP"):
        sntype = "II-norm"
    elif(sntype == "IIL"):
        sntype = "II-norm"
    elif(sntype == "Ic-SL"):
        sntype = "SLSN-I"
    elif(sntype == "SLSN-I"):
        sntype = "Ic-SLSN"
    elif(sntype == "Ia-99aa"):
        sntype = "Ia-norm"
    row.append(sntype)
    PreferredTemplatesClass.add_row(row)

In [8]:
for i in np.unique(PreferredTemplatesClass["new_type"]): print(i)

AGN
C-star
Gal
II-norm
IIb
IIn
Ia-02cx
Ia-03fg
Ia-91T
Ia-91bg
Ia-csm
Ia-norm
Ib-norm
Ibn
Ic-BL
Ic-SLSN
Ic-norm
LBV
M-star
QSO


In [9]:
counter = 0
for i in PreferredTemplatesClass:
    f = open("/home/xhall/Documents/SNID/templates-2.0/" + i["Template"] + ".lnw")
    lines = f.readlines()
    string = lines[0]
    start = string.find(i["old_type"])
    if(start != -1):
        replace = i["new_type"]
        back_string = string[start::]
        back_string_len = len(back_string)
        numlist = typeinfo[replace]
        numstring = str(numlist[0]) + "  " + str(numlist[1])
        back_string_new = str(replace) + " " * (back_string_len - len(numstring) - len(replace) - 1) + numstring + "\n"
        forward_string = string[0:start]
        document = forward_string + back_string_new
        for j in lines[1::]:
            document += j

        newfile = open("/home/xhall/Documents/SNID/templates-2.0/" + i["Template"] + ".lnw", "w+")
        newfile.write(document)
        counter += 1

In [35]:
string = "  241 1024   2500.00  10000.00     12     sn1987A       -9.99  II-pec      4  3"
start = string.find("II-pec")
replace = "II-norm"
back_string = string[start::]
back_string_len = len(back_string)
numlist = typeinfo[replace]
numstring = str(numlist[0]) + "  " + str(numlist[1])
back_string_new = str(replace) + " " * (back_string_len - len(numstring) - len(replace)) + numstring
forward_string = string[0:start]
new_string = forward_string + back_string_new

In [37]:
len(new_string)

79

In [14]:
len("\n")

1

In [10]:
for i in templates:
    f = open(i)
    lines = f.readlines()
    if(len(lines[0]) != 80):
        print(i)

In [47]:
Table.read("/home/xhall/Documents/NewZTF/SNIDoutput/ZTF17aaajnki_20180320_P60_v2_snid.output", format = "ascii", data_start = 39, names = ["no", "sn", "type", "lap", "rlap", "z", "zerr", "age", "age_flag", "grade"])

no,sn,type,lap,rlap,z,zerr,age,age_flag,grade
int64,str12,str7,float64,float64,float64,float64,float64,int64,str4
10,kcSB5,Gal,0.8786,4.68,-0.0004,0.0048,-99.9,2,good
11,kcSB3,Gal,0.8786,3.75,0.0002,0.0055,-99.9,2,good
12,kcSB6,Gal,0.8786,3.72,-0.0006,0.0056,-99.9,2,good
13,sn2004gq,Ib-norm,0.6539,3.71,0.1107,0.0181,-9.0,0,bad
14,sn04aw,Ic-norm,0.7338,3.65,0.2443,0.0089,46.0,0,bad
15,sn91T,Ia-91T,0.7852,3.62,0.1559,0.0167,-11.5,0,bad
16,sn1991T,Ia-91T,0.7852,3.61,0.1559,0.0167,-12.8,0,bad
17,sn2007C,Ib-norm,0.6552,3.6,0.0969,0.0124,-4.71,0,bad
18,sn2005az,Ic-norm,0.6525,3.58,0.2396,0.0161,22.3,0,bad
